In [1]:
import numpy as np
import netket as nk
from scripts import functions as f

/home/keisuke/Documents/Research/sshfs/DimerMaster/scripts/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [2]:
from conf import *
from numba import njit, jit

In [3]:
h = 1.0
V = 1.0

length = [8, 8]
name = 'h={}V={}l={}'.format(h, V, length)
kernel = 1
n_chains = 1
n_discards = 1
n_samples = 1e4

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)


sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70, kernel = 1)
sa.generate_samples(n_discards) # discard the begginings of metropolis sampling.
print('discard samples')
samples_state = sa.generate_samples(int(n_samples/n_chains))
samples_state = samples_state.reshape(-1, ma.hilbert.size)

print('prepared initial samples')

4
3.7305123805999756 for metropolis
discard samples
15.03051471710205 for metropolis
prepared initial samples


In [83]:
class new_dynamics:
    def __init__(self,op,ma):
        
        self.local_states = np.sort(op._local_states)
        self.basis = op._basis[::-1].copy()
        self.constant = op._constant
        self.diag_mels = op._diag_mels
        self.n_conns = op._n_conns
        self.mels = op._mels
        self.x_prime = op._x_prime
        self.acting_on = op._acting_on
        self.acting_size = np.int64(op._acting_size[0])
        self.ma = ma


        self._w = ma._w
        self._a = ma._a
        self._b = ma._b
        self._r = ma._r
        
    
    
    def dynamics(self, X, num):
        
        
        
        
        return self._dynamics(
            X, 
            num,
            self.local_states,
            self.basis,
            self.constant,
            self.diag_mels,
            self.n_conns,
            self.mels,
            self.x_prime,
            self.acting_on,
            self.acting_size,
            self._w,
        )

    
    
    @staticmethod
    @njit
    def _dynamics(
            X,
            num,
            _local_states,
            _basis,
            _constant,
            _diag_mels,
            _n_conns,
            _mels,
            _x_prime,
            _acting_on,
            _acting_size,
            _w,
            ):
        
        # basis is float64[:]
        
        
        batch_size = X.shape[0]
        L = X.shape[1]
        p_array = np.zeros((num, batch_size, L),dtype= np.int8) 
        X_float = X
        X = X.astype(np.int8)
        T = np.zeros(batch_size)
        t_array = np.zeros((num, batch_size))
        
        

        sections = np.zeros(batch_size + 1, dtype = np.int64)
        a_0 = np.zeros(batch_size)
        
        for _ in range(num):

           
            x_prime, sites, mels  = get_conn_local(
                                X,
                                sections[1:],
                                _basis,
                                _constant,
                                _diag_mels,
                                _n_conns,
                                _mels,
                                _x_prime,
                                _acting_on,
                                _acting_size)
                                



            with objmode(start='float64'):
                start = time.time()

            # log_val_prime = np.real(log_val_kernel(x_prime.astype(np.float64), None, _w, _a, _b, _r))
            R = np.empty(x_prime.shape[0], dtype=np.float64)
            for n in range(batch_size):
                R[sections[n]:sections[n+1]] = rate_psi(X_float[n], x_prime[sections[n]:sections[n+1]].astype(np.float64), sites[sections[n]:sections[n+1]], _w)

            with objmode():
                end = time.time()   
                print(end-start,'cal log')




            # for n in range(batch_size):
            #     log_val_prime[sections[n] : sections[n+1]] -= log_val_prime[sections[n]]
            

            # mels = np.real(mels) * np.exp(log_val_prime)
            mels = np.real(mels) * R

            N_conn = sections[1:] - sections[:-1]
            for n in range(batch_size):
                a_0[n] = (-1)* mels[sections[n]: sections[n+1]].sum()
#             print(a_0[0], N_conn[0], mels[sections[0] + 1: sections[1]])
                
#             print((a_0 - mels[sections[:-1]]).mean(), mels[sections[:-1]].mean())

                
            r_1 = np.random.rand(batch_size)
            r_2 = np.random.rand(batch_size)
            
            tau = np.log(1/r_1)/a_0
            t_array[_, :] = T
            T += tau

            
            p_array[_,:,:] = X
            for n in range(batch_size):
                s = 0
                for i in range(N_conn[n]):
                    s -= mels[sections[n] + i]
                    if s >= r_2[n] * a_0[n]:
                        X[n][sites[sections[n] +  i]] = x_prime[sections[n] +  i]
                        break

        return p_array, t_array
            
        
    def run(self, basis, t_list, E_0, qout):
        
        out = self.dynamics(basis, t_list, E_0)
        
        qout.put(out)
    
        
    def multiprocess(self, basis, t_list, E0 , n = 1):
        queue = []
        process = []
        N = basis.shape[0] 
        index = np.round(np.linspace(0, N, n + 1)).astype(np.int)

        for i in range(n):
            queue.append(mp.Queue())
            p = mp.Process(target=self.run, args=(basis[index[i]:index[i+1]], t_list, E0 ,queue[i]))
            p.start()
            process.append(p)


        out = []
        for q in queue:
            out.append(q.get())

        
        return np.vstack(out)

In [251]:
@njit
def rate_psi(X, X_prime_local, sites, W):
    
    n_conn = X_prime_local.shape[0]
    
    r = X.dot(W)
    tan = np.tanh(r)
#     r_prime = np.empty((n_conn, W.shape[1]), dtype=np.float64)
    out = np.empty(n_conn, dtype=np.float64)
    
    
    for n in range(n_conn):
        W_prime = W[sites[n]]
        r_prime = X_prime_local[n].dot(W_prime)
#         for j in range(W.shape[1]):
        out[n] = np.prod(np.cosh(r_prime) + tan*np.sinh(r_prime))
    return out




@jit(nopython=True)
def get_transition(
        x,
        c_r,
        s_r,
        tan,
        sections,
        basis,
        constant,
        diag_mels,
        n_conns,
        all_mels,
        all_x_prime,
        acting_on
    ):


    batch_size = x.shape[0]
    n_sites = x.shape[1]

    assert sections.shape[0] == batch_size

    n_operators = n_conns.shape[0]
    xs_n = np.empty((batch_size, n_operators), dtype=np.intp)
    xs_n_prime = np.empty(batch_size, dtype=np.intp)
    max_conn = 0

#     acting_size = np.int8(acting_size)
#     acting_size = 4
    tot_conn = 0

    sections[:] = 1

    for i in range(n_operators):
        n_conns_i = n_conns[i]
        x_i = (x[:, acting_on[i]] + 1) / 2
        s = np.zeros(batch_size)
        for j in range(4):
            s += x_i[:, j] * basis[j]
        xs_n[:, i] = s
        sections += n_conns_i[xs_n[:, i]]
    sections -= 1
    tot_conn=sections.sum()

    s = 0 
#     sec = sections.copy()
    for b in range(batch_size):
        s += sections[b]
        sections[b] = s



    x_prime = np.empty((tot_conn, 2), dtype=np.int8) # x.shpae[0] is number of connected elements of hamiltonian from batch of states. 
    t_mels = np.empty(tot_conn, dtype=np.complex128)
    sites_ = np.empty((tot_conn, 2), dtype=np.int8)
    
    log_val = np.empty(tot_conn, dtype=np.float64)
    
    acting_on = acting_on[:,1:3].copy()
    
    basis_r = np.array([3,1])
    c = 0
    
    for b in range(batch_size):
        x_batch = x[b]
        xs_n_b = xs_n[b]
#         r_b = r[b]
#         psi_b = np.prod(np.cosh(r_b))
        tan_b = tan[b]
#         psi_b = 1
#         print('psi_b',psi_b)
        
        
        for i in range(n_operators):
            
#             r_prime_i = r_primes[i]
            s_r_i = s_r[i]
            c_r_i = c_r[i]
            # Diagonal part
            n_conn_i = n_conns[i, xs_n_b[i]]

            if n_conn_i > 0:
                sites = acting_on[i]

                for cc in range(n_conn_i):
                    
                    x_prime_cc = x_prime[c + cc]
                    x_prime_cc[:] = all_x_prime[i, xs_n_b[i], cc]
                    sites_[c + cc] = sites
                    
                    num = ((np.sum((x_prime_cc - x_batch[sites]) * basis_r) + 8)/2)
#                     print(x_prime_cc - x_batch[sites])
#                     print(num)
                    sin_prime = s_r_i[np.int(num)]
                    cos_prime = c_r_i[np.int(num)]
                    
#                     print((r_prime+r_b)[10])
#                     temp = np.exp(r_prime+r_b)
#                     t_mels[c + cc] = all_mels[i, xs_n_b[i], cc] * np.prod((temp + 1/temp)/2)/psi_b# transition matrix
#                     log_val[c + cc] = np.prod(np.cosh(r_prime+r_b))/psi_b
#                     log_val[c + cc] = np.prod(tan_b*sin_prime + cos_prime)
                    t_mels[c + cc] = all_mels[i, xs_n_b[i], cc] *np.prod(tan_b*sin_prime + cos_prime)
                c += n_conn_i


    return x_prime, sites_, t_mels, log_val

In [239]:
w = ma._w
acting_list = op._acting_on[:,1:3]
w_ = w[acting_list]
x = np.zeros((9,2))

n = 0
for i in range(3):
    for j in range(3):
        
        x[n,1] = 2*(j-1)
        x[n,0] = 2*(i-1)
        n += 1

r_primes = np.einsum('ij,kjl->kil', x, w_)
c_r = np.cosh(r_primes)
s_r = np.sinh(r_primes)
r = samples_state.dot(w)
tan = np.tanh(r)

In [240]:
%timeit r = samples_state.dot(w)

1.71 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [241]:
r_ = r_primes[10,0]

In [233]:
%%timeit 
a = (np.exp(r_ + r))
# b = np.prod((a + 1/a)/2)
# a = r_ + r
# b = a + a**2 + a**4

4.78 ms ± 9.12 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [253]:
%%timeit
x_prime, sites, t_mels  = get_transition(
                            samples_state,
                            c_r,
                            s_r,
                            tan,
                            sections[1:],
                            op._basis[::-1].copy(),
                            op._constant,
                            op._diag_mels,
                            op._n_conns,
                            op._mels,
                            op._x_prime[:,:,:,1:3].copy(),
                            op._acting_on)

143 ms ± 681 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [208]:
t_mels

array([-1.00065335+0.j, -1.00042299+0.j, -1.00177615+0.j, ...,
       -0.99828775+0.j, -0.99857497+0.j, -1.00137769+0.j])

In [170]:
x = np.random.choice([-1,1],384 * 2).reshape(-1,2)
x_prime = np.random.choice([-1,1],384 * 2).reshape(-1,2)

z = x_prime - x

In [171]:
x_num = ((x + 1)/2  * np.array([2,1])).sum(axis=1)
x_prime_num = ((x_prime + 1)/2  * np.array([2,1])).sum(axis=1)
z_num = x_prime_num - x_num

In [172]:
sections = np.zeros(samples_state.shape[0] + 1, dtype=np.int)

In [212]:
%%timeit
x_prime1, mels1 = op.get_conn_flattened(samples_state,sections[1:])

84.8 ms ± 515 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [106]:
x_prime1[2].dot(ma._w)[10]

0.1561740967584369

In [79]:
(x_prime1-x_prime1[0])[2]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int8)

In [190]:
t_mels

array([-1.00065335+0.j, -1.00042299+0.j, -1.00177615+0.j, ...,
       -0.99828775+0.j, -0.99857497+0.j, -1.00137769+0.j])

In [246]:
log_prime = ma.log_val(x_prime1)

In [ ]:
log_prime 

In [247]:
np.exp(log_prime-log_prime[0]) 

array([1.        +0.j, 1.00065335+0.j, 1.00042299+0.j, ...,
       0.98759749+0.j, 0.98788163+0.j, 0.99065434+0.j])

In [250]:
log_val

array([1.00065335, 1.00042299, 1.00177615, ..., 0.99828775, 0.99857497,
       1.00137769])

In [361]:
x_prime1, mels1 = op.get_conn_flattened(samples_state, sections[1:])

In [347]:
samples_state.shape

(100, 128)

In [329]:
op._basis

array([1, 2, 4, 8])

In [26]:
d = f.dynamics2(op, ma)
import time

In [27]:
t_list = np.linspace(0, 30, 11)

In [28]:
%time d.dynamics(samples_state, t_list, 0)

594
542
548
514
484
563
516
549
546
568
521
538
529
530
519
577
553
581
560
584
555
515
493
557
543
547
542
573
537
506
538
540
514
589
535
532
535
511
546
564
507
519
508
564
549
576
556
550
496
537
542
499
497
513
612
550
534
544
535
494
561
536
527
539
538
520
597
514
551
545
578
543
544
539
535
519
506
534
556
496
576
507
520
548
543
525
572
572
534
507
548
549
505
541
550
527
543
557
567
546
CPU times: user 42 s, sys: 492 ms, total: 42.4 s
Wall time: 3.54 s


array([[[-1,  1, -1, ...,  1, -1,  1],
        [-1, -1, -1, ..., -1, -1,  1],
        [ 1,  1,  1, ..., -1, -1, -1],
        ...,
        [ 1, -1,  1, ..., -1, -1, -1],
        [ 1, -1,  1, ..., -1, -1,  1],
        [ 1,  1,  1, ..., -1,  1,  1]],

       [[-1,  1,  1, ..., -1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1],
        ...,
        [-1, -1,  1, ...,  1,  1,  1],
        [-1, -1,  1, ...,  1, -1,  1],
        [ 1,  1,  1, ...,  1,  1,  1]],

       [[-1,  1, -1, ...,  1,  1, -1],
        [-1,  1, -1, ...,  1,  1, -1],
        [ 1,  1,  1, ...,  1, -1,  1],
        ...,
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1,  1, ...,  1,  1,  1],
        [ 1,  1, -1, ..., -1,  1,  1]],

       ...,

       [[-1, -1, -1, ..., -1, -1, -1],
        [ 1, -1, -1, ..., -1, -1, -1],
        [-1, -1, -1, ..., -1, -1,  1],
        ...,
        [ 1,  1,  1, ...,  1,  1, -1],
        [-1, -1,  1, ...,  1,  1, -1],
        [-1, -1,  1, ...,  1,  1

In [299]:
@jit(fastmath=True)
def rate_psi(X, X_prime_local, sites, W):
    
    n_conn = X_prime_local.shape[0]
    
    r = X.dot(W)
    tan = np.tanh(r)
#     r_prime = np.empty((n_conn, W.shape[1]), dtype=np.float64)
    out = np.empty(n_conn, dtype=np.float64)
    
#     w = W[sites]
    for n in range(n_conn):
        r_prime = X_prime_local[n].dot(W[sites[n]])
#         for j in range(W.shape[1]):
        out[n] = np.prod(np.cosh(r_prime) + tan*np.sinh(r_prime))
    return out

In [321]:
@jit(fastmath=True)
def log_val_(X, X_prime_local, sites, W):
    
    n_conn = X_prime_local.shape[0]
    
    r = X.dot(W)
#     r_prime = np.empty((n_conn, W.shape[1]), dtype=np.float64)
    lc = logcosh(r).sum()
    
    out = np.empty(n_conn, dtype=np.float64)
    
    
    
#     w = W[sites]
    for n in range(n_conn):
        r_prime = X_prime_local[n].dot(W[sites[n]])
        f = r_prime + r
        out[n] = logcosh(f).sum()
    out -= lc
    return out

In [316]:
@njit
def logcosh(x, y):
    # s always has real part >= 0
    s = np.sign(x) * x
    p = np.exp(-2 * s)
    return s + np.log(p) - np.log(2.0)

In [296]:
x_prime = x_prime.astype(np.float64)
batch_size = samples_state.shape[0]

In [287]:
x = np.random.rand(100)

In [291]:
%timeit np.cosh(x)

961 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [290]:
%timeit (np.exp(x) + np.exp(-x))/2

2.9 µs ± 6.31 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [282]:
%%time
log_val_prime = np.empty(x_prime.shape[0], dtype=np.float64)
for n in range(batch_size):
    log_val_prime[sections[n]:sections[n+1]] = rate_psi(samples_state[n], x_prime[sections[n]:sections[n+1]], sites[sections[n]:sections[n+1]], ma._w)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float64, 2d, C), Tuple(array(int8, 2d, C), slice<a:b>))
 
There are 22 candidate implementations:
      - Of which 20 did not match due to:
      Overload of function 'getitem': File: <numerous>: Line N/A.
        With argument(s): '(array(float64, 2d, C), Tuple(array(int8, 2d, C), slice<a:b>))':
       No match.
      - Of which 2 did not match due to:
      Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 162.
        With argument(s): '(array(float64, 2d, C), Tuple(array(int8, 2d, C), slice<a:b>))':
       Rejected as the implementation raised a specific error:
         TypeError: unsupported array index type array(int8, 2d, C) in Tuple(array(int8, 2d, C), slice<a:b>)
  raised from /home/keisuke/miniconda3/envs/research/lib/python3.8/site-packages/numba/core/typing

In [223]:
n = 0

In [301]:
%%timeit
rate_psi(samples_state[n], x_prime[sections[n]:sections[n+1]], sites[sections[n]:sections[n+1]], ma._w)

40.6 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [298]:
%timeit ma.log_val(x_prime1[sections[n]:sections[n+1]])

38.7 µs ± 930 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [323]:
%timeit log_val_(samples_state[n], x_prime[sections[n]:sections[n+1]], sites[sections[n]:sections[n+1]], ma._w)

25.6 µs ± 37.8 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [111]:
ma._w[[32,24,16,8],:]

(4, 128)

In [113]:
x_prime.dot(ma._w[sites])

KeyboardInterrupt: 

In [114]:
x_prime.shape

(1766, 4)

In [209]:
import numpy as _np
@jit(nopython=True)
def get_conn_local(
        x,
        sections,
        basis,
        constant,
        diag_mels,
        n_conns,
        all_mels,
        all_x_prime,
        acting_on,
        acting_size
    ):


    batch_size = x.shape[0]
    n_sites = x.shape[1]

    assert sections.shape[0] == batch_size

    n_operators = n_conns.shape[0]
    xs_n = _np.empty((batch_size, n_operators), dtype=_np.intp)

    max_conn = 0

#     acting_size = np.int8(acting_size)
#     acting_size = 4
    tot_conn = 0

    sections[:] = 1

    for i in range(n_operators):
        n_conns_i = n_conns[i]
        x_i = (x[:, acting_on[i]] + 1) / 2
        s = _np.zeros(batch_size)
        for j in range(acting_size):
            s += x_i[:, j] * basis[j]
        xs_n[:, i] = s
        sections += n_conns_i[xs_n[:, i]]
    sections -= 1
    tot_conn=sections.sum()

    s = 0 
#     sec = sections.copy()
    for b in range(batch_size):
        s += sections[b]
        sections[b] = s



    x_prime = _np.empty((tot_conn, acting_size), dtype=_np.int8) # x.shpae[0] is number of connected elements of hamiltonian from batch of states. 
    mels = _np.empty(tot_conn, dtype=_np.complex128)
    sites_ = _np.empty((tot_conn, acting_size), dtype=_np.int8)


    c = 0
    for b in range(batch_size):
        x_batch = x[b]
        xs_n_b = xs_n[b]
        for i in range(n_operators):

            # Diagonal part
            n_conn_i = n_conns[i, xs_n_b[i]]

            if n_conn_i > 0:
                sites = acting_on[i]

                for cc in range(n_conn_i):
                    mels[c + cc] = all_mels[i, xs_n_b[i], cc]
                    x_prime_cc = x_prime[c + cc]
                    x_prime_cc[:] = all_x_prime[i, xs_n_b[i], cc] - x_batch[sites]
                    sites_[c + cc] = sites
                c += n_conn_i


    return x_prime, sites_ , mels

In [210]:
op

In [211]:
batch_size = samples_state.shape[0]
sections = np.zeros(batch_size + 1, dtype=np.int)
acting_size = int(op._acting_size[0])

In [213]:
x_prime, sites, mels =  get_conn_local(
    samples_state,
    sections[1:],
    op._basis[::-1].copy(),
    op._constant,
    op._diag_mels,
    op._n_conns,
    op._mels,
    op._x_prime,
    op._acting_on,
    acting_size
)

In [175]:
x_prime1,  mels1 = op.get_conn_flattened(samples_state, sections[1:])

In [94]:
sections

array([   0,   16,   33,   54,   70,   87,  108,  126,  144,  164,  181,
        197,  217,  235,  254,  274,  293,  313,  332,  349,  368,  387,
        406,  424,  447,  465,  487,  506,  523,  543,  562,  580,  599,
        614,  635,  655,  673,  691,  708,  723,  742,  760,  780,  800,
        821,  841,  857,  874,  894,  911,  929,  948,  965,  985, 1005,
       1019, 1036, 1055, 1069, 1094, 1114, 1132, 1151, 1171, 1189, 1210,
       1227, 1247, 1261, 1281, 1303, 1319, 1335, 1355, 1378, 1395, 1413,
       1432, 1451, 1467, 1487, 1507, 1524, 1540, 1558, 1576, 1599, 1621,
       1639, 1660, 1676, 1697, 1717, 1736, 1752, 1772, 1790, 1806, 1824,
       1842, 1866])

In [79]:
for n in range(batch_size):
    

array([   0,   16,   33,   54,   70,   87,  108,  126,  144,  164,  181,
        197,  217,  235,  254,  274,  293,  313,  332,  349,  368,  387,
        406,  424,  447,  465,  487,  506,  523,  543,  562,  580,  599,
        614,  635,  655,  673,  691,  708,  723,  742,  760,  780,  800,
        821,  841,  857,  874,  894,  911,  929,  948,  965,  985, 1005,
       1019, 1036, 1055, 1069, 1094, 1114, 1132, 1151, 1171, 1189, 1210,
       1227, 1247, 1261, 1281, 1303, 1319, 1335, 1355, 1378, 1395, 1413,
       1432, 1451, 1467, 1487, 1507, 1524, 1540, 1558, 1576, 1599, 1621,
       1639, 1660, 1676, 1697, 1717, 1736, 1752, 1772, 1790, 1806, 1824,
       1842, 1866])

In [65]:
x_prime1[1][sites[0]]

array([ 1, -1, -1,  1], dtype=int8)

In [67]:
(x_prime1[1]- x_prime1[0])[sites[0]]

array([ 0, -2, -2,  0], dtype=int8)

In [63]:
x_prime[0]

array([ 1, -1, -1,  1], dtype=int8)

In [64]:
sites

array([[ 32,  24,  16,   8],
       [ 24,  32,  47,  55],
       [ 16,   8,   1, 121],
       ...,
       [ 32,  47,  39,  46],
       [ 64,  79,  71,  78],
       [ 96, 111, 103, 110]], dtype=int8)

In [ ]:
num = 2500
batch_size = samples_state.shape[0]
P = []
for i in range(int(1)):
    s = time.time()
    print(i * num ,(i+1)*num)
    P.append(d.dynamics(samples_state[i * num : (i+1)*num], t_list, 0))
    print('{}th loop done : {}'.format(i, time.time()-s))
P = np.vstack(P)

In [ ]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


l = []
for i in range(length[0]):
    for j in range(length[1]):
        l.append([i, j])
l = np.array(l)

a_ = [a for _ in range(np.prod(length))]
a = np.array(a_)

edges, colors = hex_.dimer_corr(l,a)
operators = f.return_dimer_operator(hi, edges, colors)
sections = np.arange(P.shape[0])

_, mels1 = operators[0].get_conn_flattened(P[:,0,:], sections)
sub1 = operators[0].get_conn_flattened(P[:,0,:], sections)[1].mean().real

dimer_corr = np.zeros((length[0],length[1],t_list.shape[0]))
dimer_std = np.zeros((length[0],length[1],t_list.shape[0]))

for l1 in range(length[0]):
    for l2 in range(length[1]):
        sub2 = operators[1].get_conn_flattened(P[:,0,:], sections)[1].mean().real 
        for i in range(t_list.shape[0]):
            _, mels2 = operators[l1 * length[1] + l2].get_conn_flattened(P[:,i,:], sections)
            dimer_corr[l1,l2,i] = np.real(((mels1 * mels2 ).mean()))
            dimer_std[l1,l2,i] = np.real(((mels1 * mels2 ).std()))
        dimer_corr[l1,l2] -= sub1 * sub2
        dimer_std[l1,l2] /= np.sqrt(P.shape[0])

In [ ]:
dimer_corr2 = dimer_corr

In [ ]:
dimer_corr2[1,3]

In [ ]:
dimer_corr[1,3]

In [ ]:
dimer_std[1,3]

In [ ]:
t_list